In [1]:
# chemin de base vers le répertoire YOLO
MODEL_PATH = "yolo-coco"

# initialiser la probabilité minimale de filtrer les détections faibles avec
# le seuil lors de l'application de la suppression non maximale
MIN_CONF = 0.3
NMS_THRESH = 0.3 # suppression non maximal du seuil

# boolean indiquant si NVIDIA CUDA GPU doit être utilisé
USE_GPU = False

# Définition de la distance minimal de sécurité(en pixels) que 2 personnes peuvent être
# par rapport à l'autre
MIN_DISTANCE = 50

In [2]:
import numpy as np
import cv2

def detect_people(frame, net, ln, personIdx=0):
    # saisir les dimensions du cadre et initialiser la liste des 
    # résultats
    (H, W) = frame.shape[:2]
    results = []

    # construire un blob à partir de la trame d'entrée puis effectuer une passe avant
    # du détecteur d'objet YOLO, en nous donnant nos boxes englobantes
    # et les probabilités associées
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # initialiser nos listes de boxes englobantes détectés, centroides, et
    # confidences, respectivement
    boxes = []
    centroids = []
    confidences = []

    # Boucle sur chacune des sorties de couche
    for output in layerOutputs:
        # Boucle sur chacune des détections
        for detection in output:
            # extraire l'ID de classe et la confiance (c'est-à-dire la probabilité)
            # de la détection d'objet actuelle
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filtrer les détections en (1) s'assurant que l'objet
            # détecté était une personne et (2) que le minimum
            # de confiance est satisfaite
            if classID == personIdx and confidence > MIN_CONF:
                # redimensionne les coordonnées du cadre de sélection par rapport à
                # la taille de l'image, en gardant à l'esprit que YOLO
                # renvoie en fait les coordonnées centrales (x, y) de
                # la boxe englobante suivie de la largeur et la taille des boxes
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # utiliser les coordonnées centre (x, y) pour dériver le sommet
                # et le coin gauche du cadre de sélection
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # mettre à jour notre liste de coordonnées de la boxe englobante,
                # centroids et confidences
                boxes.append([x, y, int(width), int(height)])
                centroids.append((centerX, centerY))
                confidences.append(float(confidence))

    # appliquer une suppression non maximale pour supprimer les zones(boxe) de délimitation faibles
    # et superposées
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

    # S'assurer qu'au moins une détection existe
    if len(idxs) > 0:
        # boucle sur les index que nous gardons
        for i in idxs.flatten():
            # extraire les coordonnées de la boxe englobante
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # mettre à jour notre liste de résultats pour qu'elle soit composée de la probabilité
            # de prédiction de personne, coordonnées de la boxe englobante,
            # et le centre de gravité
            r = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(r)

    # retourne la liste des résultats
    return results

In [3]:
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

# construire le parser d'arguments et parser(analyser) les arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", type=str, default="",
	help="path to (optional) input video file")
ap.add_argument("-o", "--output", type=str, default="",
	help="path to (optional) output video file")
ap.add_argument("-d", "--display", type=int, default=1,
	help="whether or not output frame should be displayed")
args = vars(ap.parse_args(["--input","C:/Users/hp/Desktop/Social-distance-detection-master/Social-distance-detection-master/pedestrians.mp4","--output","my_output.avi","--display","1"]))

# charger les labels de classe COCO sur lesquelles notre modèle YOLO a été entrainé
labelsPath = os.path.sep.join(["C:/Users/hp/Desktop/Social-distance-detection-master/Social-distance-detection-master/yolo-coco/coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# dériver les chemins vers les poids YOLO(yolo weights) et la configuration du modèle(model configuration)
weightsPath = os.path.sep.join(["C:/Users/hp/Desktop/Social-distance-detection-master/Social-distance-detection-master/yolo-coco/yolov3.weights"])
configPath = os.path.sep.join(["C:/Users/hp/Desktop/Social-distance-detection-master/Social-distance-detection-master/yolo-coco/yolov3.cfg"])

# charger notre détecteur d'objets YOLO entrainé sur un jeu de données COCO (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# Vérifier si on va utiliser le GPU
if USE_GPU:
    # définir CUDA comme backend et cible preferé
    print("[INFO] setting preferable backend and target to CUDA...")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# déterminer uniquement les noms de couches * de sortie * dont nous avons besoin de YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# initialiser le flux vidéo et le pointeur vers le fichier vidéo de sortie
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
writer = None

# Boucle sur les images du flux vidéo
while True:
    # lire l'image suivante du fichier
    (grabbed, frame) = vs.read()

    # si le cadre n'a pas été saisi, alors nous avons atteint la fin
    # du flux
    if not grabbed:
        break

    # redimensionner le cadre, puis détecter les personnes (et uniquement les personnes) qui s'y trouvent
    frame = imutils.resize(frame, width=700)
    results = detect_people(frame, net, ln,
        personIdx=LABELS.index("person"))

    # initialiser l'ensemble des index qui violent la distance sociale minimale
    violate = set()

    # assurez-vous qu'il y a * au moins * deux détections de personnes (obligatoire
    # pour calculer nos cartes de distance par paires)
    if len(results) >= 2:
        # extraire tous les centres de gravité des résultats et calculer les
        # Distances euclidiennes entre toutes les paires de centroïdes
        centroids = np.array([r[2] for r in results])
        D = dist.cdist(centroids, centroids, metric="euclidean")

        # boucle sur le triangle supérieur de la matrice de distance
        for i in range(0, D.shape[0]):
            for j in range(i + 1, D.shape[1]):
                # vérifiez si la distance entre deux
                # paires de centroïdes est inférieur au nombre configuré
                # de pixels
                if D[i, j] < MIN_DISTANCE:
                    # mettre à jour notre jeu de violations avec les index des
                    # paires de centroïdes
                    violate.add(i)
                    violate.add(j)

    # Boucle sur les résultats
    for (i, (prob, bbox, centroid)) in enumerate(results):
        # extraire le cadre(boxe) englobant et les coordonnées du centre de gravité, puis
        # initialiser la couleur de l'annotation
        (startX, startY, endX, endY) = bbox
        (cX, cY) = centroid
        color = (0, 255, 0)

        # si la paire d'index existe dans le jeu de violations, alors
        # mettre à jour la couleur
        if i in violate:
            color = (0, 0, 255)

        # dessinez (1) un cadre de délimitation autour de la personne et (2) les
        # coordonnées centroïde de la personne,
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        cv2.circle(frame, (cX, cY), 5, color, 1)

    # dessiner le nombre total de violations de distanciation sociale sur la
    # trame(image) de sortie
    text = "Social Distancing Violations: {}".format(len(violate))
    cv2.putText(frame, text, (10, frame.shape[0] - 25),
        cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

    # vérifier si le cadre(frame - image) de sortie doit être affiché sur notre
    # écran
    if args["display"] > 0:
        # afficher la trame(image) de sortie
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # si la touche `q` a été enfoncée, sortir de la boucle
        if key == ord("q"):
            break

    # si un chemin de fichier vidéo de sortie a été fourni et que le lecteur(graveur - writer) de vidéo
    # n'a pas été initialisé, faites-le maintenant
    if args["output"] != "" and writer is None:
        # initialiser notre graveur(lecteur - writer) vidéo
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(args["output"], fourcc, 25,
            (frame.shape[1], frame.shape[0]), True)

    # si le graveur(lecteur - writer) vidéo n'est pas None, écrivez l'image sur le fichier vidéo
    # de sortie
    if writer is not None:
        writer.write(frame)


[INFO] loading YOLO from disk...
[INFO] accessing video stream...
